<a href="https://colab.research.google.com/github/Avipsa1/UPPP275-Notebooks/blob/main/Creating_Voronoi_polygons_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# install with "plotting" dependencies (recommended):
!pip install -U geovoronoi[plotting]
!pip install geopandas
!python -m pip uninstall matplotlib --y
!pip install matplotlib==3.1.3

  Using cached matplotlib-3.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Found existing installation: matplotlib 3.5.2
Uninstalling matplotlib-3.5.2:
  Successfully uninstalled matplotlib-3.5.2
  Using cached matplotlib-3.1.3-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Let us import the libraries and read the data first.

In [9]:
import geopandas as gpd
import numpy as np
import geovoronoi
import shapely

We can plot the points data. Just to give some context, we also read the boundary of the census tracts and plot the points on top of it.

In [10]:
import numpy as np
coords = np.array([[1690891.43454513, 4865911.53550427],
       [1303898.2749075 , 5398659.4816214 ],
       [1379407.32051822, 5701267.51923313],
       [1703402.05850744, 4916559.63783754],
       [1403502.05750314, 4926583.45782178]])

In [11]:
import geopandas as gpd

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
area = world[world.name == 'Italy']

Before we calculate the Voronoi regions, we need to make sure two things. First, we need to check the projection of the data and then convert it to Web Mercator projection (epsg=3395).

In [12]:
area = area.to_crs(epsg=3395)    # convert to World Mercator CRS
area_shape = area.iloc[0].geometry   # get the Polygon

Now, that we have prepared the data, we can calculate Voronoi regions simply using Geovoronoi’s method voronoi_regions_from_coords().

In [13]:
from geovoronoi import voronoi_regions_from_coords

region_polys, region_pts = voronoi_regions_from_coords(coords, area_shape)

To plot Voronoi Diagrams, we also use the functionality of Geovoronoi — plot_voronoi_polys_with_points_in_area(). Here, we provide all the output from the above Voronoi calculations and the boundary shape.

In [ ]:
import matplotlib.pyplot as plt
from geovoronoi.plotting import subplot_for_map, plot_voronoi_polys_with_points_in_area

fig, ax = subplot_for_map(figsize = (10,10))
plot_voronoi_polys_with_points_in_area(ax, area_shape, region_polys, coords, region_pts)
plt.show()

We can increase the figure size and marker size of the points. We can also change the colour using Matplotlib Colormap. 

In [ ]:
fig, ax = subplot_for_map(figsize = (10,10))
plot_voronoi_polys_with_points_in_area(ax, area_shape, region_polys, coords, region_pts, points_markersize= 120, points_marker= "*")
plt.show()

now let us try the same with some real data. Let us import the airports in SCAG region as a geojson file and create a GeoDataFrame.

In [27]:
gdf = gpd.read_file("Airports_SCAG_Region.geojson")
gdf.head()

,OBJECTID,LOC_ID,X,Y,NAME,USE_,COUNTY,YEAR,geometry
0,1,AJO,444277.416719,3.750865e+06,Corona Municipal Airport,PU,Riverside,2009,POINT (-117.60268 33.89669)
1,2,APV,482926.903875,3.826268e+06,Apple Valley Airport,PU,San Bernardino,2009,POINT (-117.18616 34.57806)
2,3,AVX,368428.577462,3.696913e+06,Catalina Airport,PU,Los Angeles,2009,POINT (-118.41488 33.40349)
3,4,BLH,711744.260243,3.722742e+06,Blythe Airport,PU,Riverside,2009,POINT (-114.71737 33.62345)
4,5,BNG,513701.841566,3.753637e+06,Banning Municipal Airport,PU,Riverside,2009,POINT (-116.85177 33.92308)


No let us import the city boundaries in SCAG region and plot the boundaries and airports together

In [ ]:
boundary = gpd.read_file("City_Boundaries_SCAG_Region.geojson")
fig, ax = plt.subplots(figsize=(12, 10))
boundary.plot(ax=ax, color="gray")
gdf.plot(ax=ax, markersize=3.5, color="red")
ax.axis("off")
plt.axis('equal')
plt.show()

We have to make sure the coordinate reference of both GeoDataFrames is Web Mercator - i.e. epsg:3395

In [29]:
boundary = boundary.to_crs(epsg=3395)
gdf_proj = gdf.to_crs(boundary.crs)

We have to convert the points into a numpy array using the points_to_coords() function from geovoronoi package.
We also combine all the city boundaries into a single large polygon using the cascaded_union () function from shapely.

In [32]:
from shapely.ops import cascaded_union
from geovoronoi import voronoi_regions_from_coords, points_to_coords

boundary_shape = cascaded_union(boundary.geometry)
coords = points_to_coords(gdf_proj.geometry)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  after removing the cwd from sys.path.


Now, calculate the voronoi regions from the coordinates of the airports. These voronoi regions could represent the service areas of each of the airports in the SCAG region.

In [33]:
region_polys, region_pts = voronoi_regions_from_coords(coords, boundary_shape)

Finally, plot the voronoi regions with the individual airport locations.

In [ ]:
fig, ax = subplot_for_map(figsize = (10,10))
plot_voronoi_polys_with_points_in_area(ax, boundary_shape, region_polys, coords, region_pts, points_markersize= 120, points_marker= "*")
plt.show()